## Classification Models Notebook
This notebook tests different types of sentiment analysis / classification models, namely TextBlob, VADER, DistilBERT, and Lexicon-Based.

In [ ]:
from textblob import TextBlob
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer
from transformers import pipeline

D:\Anaconda\envs\CS426\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


## Define Test Dataset
This data was randomly selected from the entire pool of patch notes. The data was manually labeled as either a nerf (0), buff (1), or neutral (2) to serve as the ground truth to calculate classification accuracy.

In [2]:
test_labels = [0, 0, 2, 0, 1, 2, 2, 2, 1, 2, 0, 1, 2, 2, 0, 2, 0, 1, 2, 2, 1, 0, 1, 1, 1, 1, 2, 2, 1, 1, 2, 2, 2, 2, 2,
              1, 0, 0, 1, 2, 1, 1, 2, 2, 2, 1, 1, 2, 2]
test_text = [
    "Shield AP ratio reduced to 45% AP from 50% AP.",
    "Base health regeneration reduced to 3.75 from 6.",
    "This never affected gameplay.",
    "H-28Q Apex Turret Base health changed to 725 \u2212 1525 (based on level) from 850 \u2212 1450 (based on level).",
    "Second cast AD ratio increased to 75 / 87.5 / 100 / 112.5 / 125% AD from 75 / 81.25 / 87.5 / 93.75 / 100% AD.",
    "Scaling changed to (+ 10% AP) (+ 10% armor) from (+ 15% AP).",
    "Bug Fix: Now uses the correct animation for the attack rather than the default one when he has any passive stacks.",
    "New in-game quest with  Senna.",
    "Missile speed increased to 850 from 750.",
    "Bug Fix: Now properly breaks spell shields.",
    "Base damage reduced to 70 / 110 / 150 / 190 / 230 from 80 / 120 / 160 / 200 / 240.",
    "Cooldown reduced to 12 / 11.5 / 11 / 10.5 / 10 seconds from 12 at all ranks.",
    "Nautilus slams the ground, causing the earth to ripple out in waves.",
    "Bug Fix: Tower creation SFX and VFX no longer incorrectly play repeatedly upon spawning the tower.",
    "Cooldown increased to 150 / 120 / 90 seconds from 120 / 90 / 60.",
    "Her basic attacks each consume 1 stack.",
    "Self bonus movement speed AP ratio reduced to 3% per 100 AP from 4% per 100 AP.",
    "Can now be primed while Karma is disabled.",
    "Fixed a bug where Requiem's team ult HUD indicator would display as available even when it wasn't.",
    "The cooldown now begins when the ability is successfully cast.",
    "Base magic resistance increased to 32.1 from 30.",
    "Explosion AP ratio reduced to 70% AP from 100% AP.",
    "Increased resistances increased to 12% from 10%.",
    "Base resistances increased to 20 / 70 / 120 (based on level) from 20 / 60 / 100 (based on level).",
    "Ability power ratio increased to 0.75 from 0.6.",
    "Now slows the enemy champion by 99% for 0.25 seconds.",
    "Cooldown: 14 seconds.",
    "Bug Fix: Fixed a bug where she was able to recast it on  Gwen while she was in her  Hallowed Mist.",
    "Non-champion AD ratio increased to 175% AD from 150% AD.",
    "Thrust range increased by 25 (to 560 from 535).",
    "New missiles and hits Now has a small cast effect.",
    "Bug Fix: Now correctly trigger  Overheal.",
    "Bonus mana regeneration while stealthed changed to +2% missing mana per second from +1% maximum mana per second.",
    "If he hits an enemy, he'll drag himself to his target and his target to him.",
    "New dash effect when Nocturne casts the ability.",
    "Cooldown reduced to 11 / 10 / 9 / 8 / 7 seconds from 12 / 11 / 10 / 9 / 8.",
    "Cast range reduced to 600 from 625.",
    "Minimum base damage per hit reduced to 8 / 10 / 12 / 14 / 16 from 8 / 11 / 14 / 17 / 20.",
    "Health growth increased to 104 from 90.",
    "New Effect: Camera now zooms out a bit when casting this spell.",
    "New Effect: While Aflame and Exalted, or Transcendent, the distance of the waves launched by her basic attacks now scales with her bonus attack range.",
    "Base health increased to 588 from 518.",
    "Old Active: Annie wraps herself and  Tibbers in a fiery aura, gaining 30% \u2212 60% (based on level) bonus movement speed that decays over 1.5 seconds, and reducing damage taken by 13 / 17 / 21 / 25 / 29% for 3 seconds.",
    "New Effect: Now draws nearby minion aggro when targeting an enemy champion.",
    "New ability icons.",
    "Base mana increased to 400 from 377.",
    "Mana cost reduced to 100 from 150.",
    "Updated tooltips.",
    "You will also see a self-only targeting particle when using this ability, depicting what command your pet is following.",
]

## Classification Accuracy Calculation

In [3]:
def calculate_accuracy(labels, ignore=False):
    correct_count = 0
    ignore_count = 0
    for i in range(len(labels)):
        if ignore and test_labels[i] == 2:
            ignore_count += 1
            continue

        if labels[i] == test_labels[i]:
            correct_count += 1

    return correct_count / (len(labels) - ignore_count)

## TextBlob

In [4]:
labels = []
for data in test_text:
    # Create a TextBlob object
    blob = TextBlob(data)

    # Analyze sentiment
    sentiment_score = blob.sentiment.polarity

    # Classify sentiment
    if sentiment_score > 0:
        labels.append(1)
    elif sentiment_score < 0:
        labels.append(0)
    else:
        labels.append(2)

print(calculate_accuracy(labels))

0.32653061224489793


## VADER

In [5]:
labels = []
for data in test_text:
    # Initialize the VADER sentiment analyzer
    analyzer = SentimentIntensityAnalyzer()

    # Analyze sentiment
    scores = analyzer.polarity_scores(data)

    if scores['compound'] > 0:
        labels.append(1)
    elif scores['compound'] < 0:
        labels.append(0)
    else:
        labels.append(2)

print(calculate_accuracy(labels))

0.5102040816326531


## DistilBERT

In [6]:
sentiment_pipeline = pipeline("sentiment-analysis", model="distilbert-base-uncased-finetuned-sst-2-english")

labels = []
for data in test_text:
    sentiment = sentiment_pipeline(data)

    if sentiment[0]['label'] == "POSITIVE":
        labels.append(1)
    elif sentiment[0]['label'] == "NEGATIVE":
        labels.append(0)
    else:
        labels.append(2)

print(calculate_accuracy(labels, True))

D:\Anaconda\envs\CS426\lib\site-packages\huggingface_hub\file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


0.6923076923076923


## Lexicon-Based

In [7]:
def get_labels(texts):
    # Keywords where if increased, it is positive.
    # Note: Require spaces before/after "ad" in case of certain words (e.g., "ADjusted" or "insteAD")
    norm = ['health', 'damage', 'movement speed', 'mana growth', 'mana per level', 'base mana', 'mana regeneration', 'mana restored', 'attack speed', ' ad ', ' ap ', 'ap scaling', 'duration', 'shield strength', 'heal', 'range', 'armor', 'lifesteal', 'life steal', 'spell vamp', 'slow', 'ability power', 'radius', 'magic reduction', 'gold', 'speed', 'stacks', 'per stack', 'magic resistance', 'hitbox']

    # Keywords where if increased, it is negative.
    inv_norm = ['mana cost', 'cooldown', 'shield decay', 'cost', 'reload time', 'windup', 'timer', 'delay', 'cast time']

    # Keywords for neutral changes.
    neutral = ['bug', 'fixed', 'new', 'changed', 'now', 'animation', 'updated', 'added', 'improved']

    # Positive keywords.
    positive = ['increased']

    # Negative keywords.
    negative = ['reduced', 'decreased', 'removed']

    data = []
    labels = []
    for text in texts:
        text = text.lower()
        # Important to check for neutral keywords (e.g., "bux") first since patch might include norm/inv_norm keywords.
        if any(stat in text for stat in neutral):
            label = 2
            data.append({'label': label, 'text': text})
            labels.append(label)
        # Grant will likely be positive.
        elif "grants" in text:
            label = 1
            data.append({'label': label, 'text': text})
            labels.append(label)
        elif any(stat in text for stat in norm):
            if any(key in text for key in positive):
                label = 1
                data.append({'label': label, 'text': text})
                labels.append(label)
            elif any(key in text for key in negative):
                label = 0
                data.append({'label': label, 'text': text})
                labels.append(label)
            else:
                label = 1
                data.append({'label': label, 'text': text})
                labels.append(label)
        elif any(stat in text for stat in inv_norm):
            if any(key in text for key in positive):
                label = 0
                data.append({'label': label, 'text': text})
                labels.append(label)
            elif any(key in text for key in negative):
                label = 1
                data.append({'label': label, 'text': text})
                labels.append(label)
            else:
                label = 0
                data.append({'label': label, 'text': text})
                labels.append(label)
        else:
            label = 2
            data.append({'label': label, 'text': text})
            labels.append(label)

    return labels, data


labels, data = get_labels(test_text)

print(calculate_accuracy(labels))

0.8367346938775511
